# [초급 프로젝트] 4팀_김명환

---
---

# 프로그래밍

In [ ]:
!pip install -q gdown
!pip install -q albumentations
!pip install -q ultralytics
!pip install -q -U ultralytics
!pip install -q nbformat
!pip install -q roboflow
!pip install -q opencv-python
!pip install -q opencv-python-headless
!pip install -q wandb
print("로딩완료")

In [ ]:
!wandb login 86a7b8c07184b2efdfb116546a17b1905e41cb5d

In [ ]:
# 기본 라이브러리 (중복 제거 및 정리)

# --- Scikit-learn: 데이터 전처리, 모델, 평가 ---
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import (
    fetch_california_housing, load_iris, make_moons, make_circles,
    load_breast_cancer, load_wine
)
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, average_precision_score

# --- 이미지 처리 ---
import cv2
from PIL import Image, ImageFilter, ImageDraw
import albumentations as A

# --- PyTorch: 딥러닝 관련 ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset
# 문제 있는 v2 import 제거하고 필요시에만 개별적으로 import
# from torchvision.transforms import v2, functional as TF
from torchvision.transforms import functional as TF
from torchvision.datasets import CocoDetection
from torch.nn import CrossEntropyLoss
from collections import OrderedDict

# --- COCO 데이터셋 관련 ---
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask

# --- 딥러닝 모델 ---
import timm

# --- 기본 라이브러리 ---
import os
import sys
import re
import csv
import copy
import json
import math
import random
import yaml
import shutil
import requests
import xml.etree.ElementTree as ET
from pathlib import Path

# --- 데이터 분석 및 시각화 ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# --- 시간 관련 ---
from datetime import datetime, timezone, timedelta
import pytz

# --- 진행률 표시 ---
import IPython.display
from tqdm.notebook import tqdm

# --- 시간대 설정 ---
__kst = pytz.timezone('Asia/Seoul')

# --- GPU 설정 ---
__device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
__device_cpu = torch.device('cpu')

# --- 재현 가능한 결과를 위한 시드 설정 ---
np.random.seed(42)
torch.manual_seed(42)
if __device.type == 'cuda':
    torch.cuda.manual_seed_all(42)

print(f"라이브러리 로드 완료 사용장치: {__device}")

In [ ]:
from urllib.request import urlretrieve; urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_c0z0c_dev.py", "helper_c0z0c_dev.py")
import importlib
import helper_c0z0c_dev as helper
importlib.reload(helper)

In [ ]:
import os, sys
from pathlib import Path

utils_dir = None
if helper.is_colab:
    utils_dir = "/content/drive/MyDrive/codeit_ai_health_eat/src/python_modules/utils"
else:
    utils_dir = os.path.join(Path.cwd().drive + '\\', 'GoogleDrive', "codeit_ai_health_eat", "src", "python_modules", "utils")

print("utils_dir:", utils_dir)

sys.path.append(str(utils_dir))
print("sys.path:", sys.path)
import importlib
import health_ea_utils as heu
importlib.reload(heu)
from health_ea_utils import *

print("helper.__file__:", helper.__file__)
print("health_ea_utils.__file__:", heu.__file__)


# 1. 학습용 데이타 다운로드 및 압축 풀기

In [ ]:
def get_tqdm_kwargs():
    """Widget 오류를 방지하는 안전한 tqdm 설정"""
    return {
        'disable': False,
        'leave': True,
        'file': sys.stdout,
        'ascii': True,  # ASCII 문자만 사용
        'dynamic_ncols': False,
#        'ncols': 80  # 고정 폭
    }

def drive_root():
    root_path = os.path.join("D:\\", "GoogleDrive")
    if helper.is_colab:
        root_path = os.path.join("/content/drive/MyDrive")
    return root_path

def get_path_modeling(add_path = None):
    modeling_path = "modeling_yolo"
    path = os.path.join(drive_root(),modeling_path)
    if add_path is not None:
        path = os.path.join(path,add_path)
    return path

def get_path_modeling_release(add_path = None):
    modeling_path = "modeling_yolo"
    path = os.path.join(drive_root(),modeling_path)
    if add_path is not None:
        path = os.path.join(path,add_path)
    return path

def print_dir_tree(root, max_depth=2, list_count=3, indent=""):
    import os
    if max_depth < 0:
        return
    try:
        items = os.listdir(root)
    except Exception as e:
        print(indent + f"[Error] {e}")
        return

    img_count = len([f for f in os.listdir(root) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt'))])
    for item in items:
        path = os.path.join(root, item)
        if os.path.isdir(path):
            print(indent + "|-- "+ item)
            # 이미지 파일 개수만 출력
            img_count = len([f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt'))])
            if img_count > list_count:
                print(indent + "   "+ f"[데이터파일: {img_count}개]")
            print_dir_tree(root=path, max_depth=max_depth-1, list_count=list_count, indent=indent + "   ")
        else:
            if list_count < img_count and item.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt')):
                continue
            print(indent + "|-- "+ item)

def save_model_dict(model, path, pth_name, kwargs=None):
    """모델 state_dict와 추가 정보를 저장"""
    def safe_makedirs(path):
        """안전한 디렉토리 생성"""
        if os.path.exists(path) and not os.path.isdir(path):
            os.remove(path)  # 파일이면 삭제
        os.makedirs(path, exist_ok=True)

    # 디렉토리 생성
    safe_makedirs(path)

    # 모델 구조 정보 추출
    model_info = {
        'class_name': model.__class__.__name__,
        'init_args': {},
        'str': str(model),
        'repr': repr(model),
        'modules': [m.__class__.__name__ for m in model.modules()],
    }

    # 생성자 인자 자동 추출(가능한 경우)
    if hasattr(model, '__dict__'):
        for key in ['in_ch', 'base_ch', 'num_classes', 'out_ch']:
            if hasattr(model, key):
                model_info['init_args'][key] = getattr(model, key)

    # kwargs 처리
    extra_info = {}
    if kwargs is not None:
        if isinstance(kwargs, str):
            extra_info = json.loads(kwargs)
        elif isinstance(kwargs, dict):
            extra_info = kwargs

    model_info.update(extra_info)

    # 저장할 dict 구성
    save_dict = {
        'model_state': model.state_dict(),
        'class_name': model.__class__.__name__,
        'model_info': model_info,
    }

    save_path = os.path.join(path, f"{pth_name}.pth")
    torch.save(save_dict, save_path)
    return save_path

def load_model_dict(path, pth_name=None):
    """
    save_model_dict로 저장한 모델을 불러오는 함수
    반환값: (model_state, model_info)
    """
    import torch
    load_path = path
    if pth_name is not None:
        load_path = os.path.join(path, f"{pth_name}.pth")
    checkpoint = torch.load(load_path, map_location='cpu', weights_only=False)  # <-- 여기 추가
    model_state = checkpoint.get('model_state')
    model_info = checkpoint.get('model_info')
    model_info['file_name'] = os.path.basename(load_path)
    return model_state, model_info


def search_pth_files(base_path):
    """
    입력된 경로의 하위 폴더들에서 pth 파일들을 검색
    """
    pth_files = []

    if not os.path.exists(base_path):
        print(f"경로가 존재하지 않습니다: {base_path}")
        return pth_files

    print(f"pth 파일 검색 시작: {base_path}")

    # 하위 폴더들을 순회하며 pth 파일 검색
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.pth'):
                pth_path = os.path.join(root, file)
                pth_files.append(pth_path)

    # 결과 정리 및 출력
    if pth_files:
        print(f"\n발견된 pth 파일들 ({len(pth_files)}개):")
        for i, pth_file in enumerate(pth_files, 1):
            # 상대 경로로 표시 (base_path 기준)
            rel_path = os.path.relpath(pth_file, base_path)
            print(f" {i:2d}. {rel_path}")
    else:
        print("pth 파일을 찾을 수 없습니다.")

    return pth_files

def print_json_tree(data, indent="", max_depth=4, _depth=0, list_count=2, print_value=True):
    """
    JSON 객체를 지정한 단계(max_depth)까지 트리 형태로 출력
    - list 타입은 3개 이상일 때 개수만 출력
    - 하위 노드가 값일 경우 key(type) 형태로 출력
    - print_value=True일 때 key(type): 값 형태로 출력
    """
    if _depth > max_depth:
        return
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                print(f"{indent}|-- {key}")
                print_json_tree(value, indent + "    ", max_depth, _depth + 1, list_count, print_value)
            else:
                if print_value:
                    print(f"{indent}|-- {key}({type(value).__name__}): {value if len(str(value)) < 100 else f'{str(value)[:30]}...'}")
                else:
                    print(f"{indent}|-- {key}({type(value).__name__})")
    elif isinstance(data, list):
        if len(data) > list_count:
            print(f"{indent}|-- [list] ({len(data)} items)")
        else:
            for i, item in enumerate(data):
                if isinstance(item, (dict, list)):
                    print(f"{indent}|-- [{i}]")
                    print_json_tree(item, indent + "    ", max_depth, _depth + 1, list_count, print_value)
                else:
                    if print_value:
                        print(f"{indent}|-- [{i}]({type(item).__name__}): {item if len(str(item)) < 100 else f'{str(item)[:30]}...'}")
                    else:
                        print(f"{indent}|-- [{i}]({type(item).__name__})")
    else:
        if print_value:
            print(f"{indent}{type(data).__name__}: {data if len(str(data)) < 100 else f'{str(data)[:30]}...'}")
        else:
            print(f"{indent}{type(data).__name__}")

def print_git_tree(data, indent="", max_depth=3, _depth=0):
    """
    PyTorch tensor/딕셔너리/리스트를 git tree 스타일로 출력
    """
    import torch
    import numpy as np

    if _depth > max_depth:
        return
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{indent}├─ {key} [{type(value).__name__}]")
            print_git_tree(value, indent + "│  ", max_depth, _depth + 1)
    elif isinstance(data, (list, tuple)):
        for i, item in enumerate(data):
            print(f"{indent}├─ [{i}] [{type(item).__name__}]")
            print_git_tree(item, indent + "│  ", max_depth, _depth + 1)
    elif torch.is_tensor(data):
        shape = tuple(data.shape)
        dtype = str(data.dtype)
        preview = str(data)
        preview_str = preview[:80] + ("..." if len(preview) > 80 else "")
        print(f"{indent}└─ Tensor shape={shape} dtype={dtype} preview={preview_str}")
    elif isinstance(data, np.ndarray):
        shape = data.shape
        dtype = data.dtype
        preview = str(data)
        preview_str = preview[:80] + ("..." if len(preview) > 80 else "")
        print(f"{indent}└─ ndarray shape={shape} dtype={dtype} preview={preview_str}")
    else:
        val_str = str(data)
        print(f"{indent}└─ {type(data).__name__}: {val_str[:80]}{'...' if len(val_str)>80 else ''}")


print("유틸리티 함수 로드 완료")

# 데이타 다운로드

In [ ]:
# download_files={
#     'yolo_label_one_class' : r'https://drive.google.com/file/d/177_86k4BuT6JnFnq7ZHJtEjp7jaRbCl2/view?usp=sharing',
#     'yolo_label' : r'https://drive.google.com/file/d/1nc-WFcw7lCS7s7VGzN9Kxh80PiBBggez/view?usp=sharing',
#     'yolo_resize_one_class' : r'https://drive.google.com/file/d/1Ak0EvkMnuwvcAFvTO-zovIgVcNlROjsS/view?usp=sharing',
#     'yolo_resize' : r'https://drive.google.com/file/d/1kpo57qOJhEhrkuzUCEh57ILB5xSPVoFv/view?usp=sharing',
# }

# download_files={
#     'yolo_label' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODU5OTkzNTMyOHxGfDA&svcType=MYBOX-WEB&time=1757776010785',
#     'yolo_label_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODYzOTg5NDExMnxGfDA&svcType=MYBOX-WEB&time=1757776673721',
#     'yolo_resize_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODgwNjk2NDMyMHxGfDA&svcType=MYBOX-WEB&time=1757780142635',
#     'yolo_resize' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODY4MDc2MjQ2NHxGfDA&svcType=MYBOX-WEB&time=1757780177672',
# }

download_files={
    # 'yolo_label' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODU5OTkzNTMyOHxGfDA&svcType=MYBOX-WEB&time=1757776010785',
    # 'yolo_label_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODYzOTg5NDExMnxGfDA&svcType=MYBOX-WEB&time=1757776673721',
    # 'yolo_resize_one_class' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODgwNjk2NDMyMHxGfDA&svcType=MYBOX-WEB&time=1757780142635',
    # 'yolo_resize' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc1ODY4MDc2MjQ2NHxGfDA&svcType=MYBOX-WEB&time=1757780177672',
    'yolo_noresize' : r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDA0ODY2ODI1NnxGfDA&svcType=MYBOX-WEB&time=1757851996107',
    #'yolo_noresize_one_class':r'https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDgwMDcxODk0NHxGfDA&svcType=MYBOX-WEB&time=1757893856220',
}
# yolo_noresize = https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDA0ODY2ODI1NnxGfDA&svcType=MYBOX-WEB&time=1757851996107
# yolo_noresize_one_class = https://fs.mybox.naver.com/file/download.api?resourceKey=YzB6MGN8MzQ3MjU5Nzc2NDgwMDcxODk0NHxGfDA&svcType=MYBOX-WEB&time=1757893856220



In [ ]:
import gdown
def download_gdrive_file(url, output_path, ignore=True):
    # 공유 링크에서 파일 ID 추출
    if os.path.exists(output_path):
        if ignore:
            os.remove(output_path)
        else:
            return

    file_id_match = re.search(r'/d/([a-zA-Z0-9_-]+)', url)
    if not file_id_match:
        raise ValueError("Google Drive 파일 ID를 찾을 수 없습니다.")
    file_id = file_id_match.group(1)
    gdown.download(f"https://drive.google.com/uc?id={file_id}", output_path, quiet=False)

def download_http(url, target, ignore=True):
    """
    HTTP 파일 다운로드 함수 (진행률 표시)
    url: 다운로드할 파일 URL
    target: 저장할 파일 경로
    ignore: True면 기존 파일 삭제 후 다운로드, False면 파일 있으면 건너뜀
    """
    if os.path.exists(target):
        if ignore:
            os.remove(target)
        else:
            print(f"이미 파일이 존재합니다: {target}")
            return target

    response = requests.get(url, stream=True)
    total = int(response.headers.get('content-length', 0))
    with open(target, 'wb') as file, tqdm(
        desc=f"Downloading {os.path.basename(target)}",
        total=total,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
        ascii=True
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            size = file.write(data)
            bar.update(size)
    print(f"다운로드 완료: {target}")
    return target

# local_code_it_ai04 = os.path.join( '~/.cache/' if helper.is_colab else Path.cwd().drive + '\\'
#                                   ,'temp'
#                                   , 'code_it_ai04')

if helper.is_colab:
    local_code_it_ai04 = os.path.join( '/content/', 'code_it_ai04')
else:
    local_code_it_ai04 = os.path.join( Path.cwd().drive + '\\', 'temp', 'code_it_ai04')

print("local_code_it_ai04:", local_code_it_ai04)

os.makedirs(local_code_it_ai04, exist_ok=True)  # 폴더 생성 코드 추가
unzip_paths = []
for key, url in download_files.items():
    print(f"{key}: {url}")
    zipfile = os.path.join(local_code_it_ai04, f'{key}.zip')
    unzip_path = os.path.join(local_code_it_ai04, f'{key}.zip.unzip')
    if os.path.exists(unzip_path):
        print(f"이미 압축해제된 폴더가 존재합니다: {unzip_path}")
        print('unzipfile:', unzip_path)
        unzip_paths.append(unzip_path)
        continue
    #download_gdrive_file(url, os.path.join(local_code_it_ai04, f'{key}.zip'), ignore=False)
    download_http(url, zipfile, ignore=False)
    unzip_path_list = heu.unzip([os.path.join(local_code_it_ai04, f'{key}.zip')])
    # for p in unzip_path_list:
    #     unzip_paths.append(p)
    print('unzip_path_list:', unzip_path_list)
    unzip_paths.extend(unzip_path_list)


### > 설정 < 플레그

In [ ]:
# google drive root에 keggle.json 파일 필요합니다.
for path in unzip_paths:
    print("압축해제된 폴더:", path)

yolo_dataset_path = os.path.join(local_code_it_ai04, f'yolo_label_one_class.zip.unzip')
yaml_path = os.path.join(yolo_dataset_path, "dataset.yaml")

def get_path_data():
    path = yolo_dataset_path
    return path

print("yaml_path:", yaml_path)
print("get_path_data:", get_path_data())

## YOLO 모델링

In [ ]:
from pathlib import Path
from ultralytics import YOLO

# GPU 확인
device = __device
print(f"사용 디바이스: {device}")
print(f"CUDA 버전: {torch.version.cuda}")

### 3.3 모델링 엔진

In [ ]:
for path in unzip_paths:
    print(f"압축해제된 폴더: {path}")


In [ ]:
def extract_folder_key(folder_path):
    """
    폴더 경로에서 yolo_label_one_class 등 주요 명칭만 추출
    예시: 'd:\\temp\\code_it_ai04\\yolo_label_one_class.zip.unzip' → 'yolo_label_one_class'
    """
    # 경로에서 마지막 폴더/파일명 추출
    base = os.path.basename(folder_path)
    # .zip 또는 .zip.unzip 등 확장자 제거
    key = re.sub(r'\.zip(\.unzip)?$', '', base)
    return key

def update_yaml_paths_to_absolute(yaml_path):
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    yaml_dir = os.path.dirname(yaml_path)
    data['path'] = os.path.normpath(os.path.join(yaml_dir, data['path']))
    # for key in ['train', 'val', 'test']:
    #     if key in data and not os.path.isabs(data[key]):
    #         data[key] = os.path.normpath(os.path.join(yaml_dir, data[key]))

    with open(yaml_path, 'w') as f:
        yaml.dump(data, f, allow_unicode=True)

import wandb
wandb.login()  # 또는 wandb.init(project="your_project_name")
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False


# TEST

In [ ]:
yolo_dataset_path =unzip_paths[0]
yaml_path = os.path.join(yolo_dataset_path, "dataset.yaml")

print(unzip_paths)
print(yolo_dataset_path)
print(yaml_path)

# /content/code_it_ai04/yolo_noresize.zip.unzip/dataset.yaml

with open(yaml_path, 'r') as f:
    dataset_config = yaml.safe_load(f)
test_images = [str(p) for p in Path(os.path.join(yolo_dataset_path, dataset_config['test'])).glob("*.png")]
test_images = [str(p) for p in Path(os.path.join(yolo_dataset_path, dataset_config['test'])).glob("*.jpg")]

In [ ]:
print(test_images)

In [ ]:
model_1way_path = os.path.join(get_path_modeling(),
                               'yolov8m_yolo_noresize_one_class_20250915_0858',
                               'weights','best.pt')
model_2way_path = os.path.join(get_path_modeling(),
                               'efficientnet_b3_experiment_20250915_211208',
                               'best.pth')

print("model_1way_path:", model_1way_path, os.path.exists(model_1way_path))
print("model_2way_path:", model_2way_path, os.path.exists(model_2way_path))

In [ ]:
CLASS_NAMES = [1899, 2482, 3350, 3482, 3543, 3742, 3831, 4377, 4542, 5093, 5885, 6191, 6562, 10220, 12080, 12246, 12419, 12777, 13394, 13899, 16231, 16261, 16547, 16550, 16687, 18109, 18146, 18356, 19231, 19551, 19606, 19860, 20013, 20237, 20876, 21025, 21324, 21770, 22073, 22346, 22361, 22626, 23202, 23222, 24849, 25366, 25437, 25468, 27652, 27732, 27776, 27925, 27992, 28762, 29344, 29450, 29666, 29870, 30307, 31704, 31862, 31884, 32309, 33008, 33207, 33877, 33879, 34596, 35205, 36636, 38161, 41767, 44198]
NUM_CLASSES = len(CLASS_NAMES)

In [ ]:
model_1way = YOLO(model_1way_path)
model_1way.to(__device)
model_1way.eval();

In [ ]:
model_2way_state, model_2way_info = load_model_dict(model_2way_path)
model_2way = timm.create_model("efficientnet_b3", pretrained=True, num_classes=NUM_CLASSES)
model_2way.load_state_dict(model_2way_state)
model_2way.to(__device)
model_2way.eval();

In [ ]:
os.environ['YOLO_VERBOSE'] = 'False'
os.environ['ULTRALYTICS_LOG_LEVEL'] = 'WARNING'  # 또는 'ERROR'

In [ ]:
transform = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

def search_class_idx(img, box):
    xyxy = box.xyxy[0].cpu().numpy().astype(int)
    x1, y1, x2, y2 = xyxy
    image = Image.fromarray(img[y1:y2, x1:x2])  # numpy 슬라이싱으로 자르기
    input_tensor = transform(image).unsqueeze(0).to(__device)
    output = model_2way(input_tensor)
    _, predicted = torch.max(output, 1)
    cls_idx = predicted.item()
    predicted_class = CLASS_NAMES[cls_idx]
    return cls_idx, predicted_class, xyxy

In [ ]:
print(test_images[0])
result = model_1way(test_images[0], verbose=False)

In [ ]:
import matplotlib.patches as patches

img = result[0].orig_img.copy()
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(img)

for box in result[0].boxes:
    cls_idx, predicted_class, xyxy = search_class_idx(img, box)
    print(f"Predicted class : [{cls_idx}]  {predicted_class} {xyxy}")
    xyxy = box.xyxy[0].cpu().numpy().astype(int)
    cls = int(box.cls.item())
    conf = box.conf.item()
    label = f"{predicted_class} {conf:.2f}"
    color = 'red'  # 빨강색
    x1, y1, x2, y2 = xyxy
    # int()로 확실하게 정수 변환
    rect = patches.Rectangle((int(x1), int(y1)), int(x2-x1), int(y2-y1),
                           linewidth=2, edgecolor=color, facecolor='none')
    print(f'[{cls}] {predicted_class} {xyxy} {conf:.2f}')
    ax.add_patch(rect)
    ax.text(int(x1), int(y1)-5, label, color=color, fontsize=12, backgroundcolor='black')

plt.axis('off')
plt.show()

In [ ]:
def save_detection_image(image_path, save_folder):
    """이미지에서 약물을 검출하고 bbox와 label을 그려서 저장"""
    result = model_1way(image_path, verbose=False)
    
    # 이미지 ID (파일명에서 추출)
    image_id = os.path.splitext(os.path.basename(image_path))[0]
    
    # 해당 이미지의 실제 numpy 배열 가져오기
    current_img = result[0].orig_img.copy()
    
    # matplotlib으로 이미지 그리기
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.imshow(current_img)
    
    detection_count = 0
    
    for box in result[0].boxes:
        cls_idx, predicted_class, xyxy = search_class_idx(current_img, box)
        
        # YOLO bbox 좌표
        x1, y1, x2, y2 = xyxy
        conf = box.conf.item()
        
        # 바운딩 박스 그리기
        rect = patches.Rectangle((int(x1), int(y1)), int(x2-x1), int(y2-y1),
                               linewidth=2, edgecolor='red', facecolor='none')
        ax.add_patch(rect)
        
        # 라벨 텍스트
        label = f"{predicted_class} {conf:.2f}"
        ax.text(int(x1), int(y1)-5, label, color='red', fontsize=10, 
                backgroundcolor='white', alpha=0.8)
        
        detection_count += 1
    
    # 축과 여백 제거
    ax.set_xlim(0, current_img.shape[1])
    ax.set_ylim(current_img.shape[0], 0)  # y축 뒤집기
    ax.axis('off')
    plt.tight_layout(pad=0)
    
    # 저장 경로
    save_path = os.path.join(save_folder, f"{image_id}_detected.jpg")
    
    # 이미지 저장
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0, dpi=150, format='jpg')
    plt.close()  # 메모리 절약
    
    return save_path, detection_count

def save_all_detection_images():
    """모든 테스트 이미지에 대해 검출 결과를 이미지로 저장"""
    
    # 결과 저장 폴더 생성
    timestamp = datetime.now(__kst).strftime("%Y%m%d_%H%M%S")
    result_folder_name = f'result_{timestamp}'
    
    # output_path 설정 (기존 csv_path와 같은 위치)
    output_folder = os.path.join(drive_root(),
                                'codeit_ai_health_eat',
                                'scripts',
                                '김명환',
                                result_folder_name)
    
    # 폴더 생성
    os.makedirs(output_folder, exist_ok=True)
    
    print(f"검출 결과 이미지 저장 시작...")
    print(f"저장 폴더: {output_folder}")
    
    saved_images = []
    total_detections = 0
    
    for i, image_path in enumerate(tqdm(test_images, desc="이미지 검출 및 저장")):
        try:
            # 검출 결과 이미지 저장
            saved_path, detection_count = save_detection_image(image_path, output_folder)
            saved_images.append(saved_path)
            total_detections += detection_count
            
        except Exception as e:
            print(f"이미지 저장 실패 {image_path}: {e}")
            continue
    
    print(f"\n검출 결과 이미지 저장 완료:")
    print(f"- 처리된 이미지: {len(saved_images)}개")
    print(f"- 총 검출 객체: {total_detections}개")
    print(f"- 저장 폴더: {output_folder}")
    
    return output_folder, saved_images

# 실행
output_path, saved_images = save_all_detection_images()

In [ ]:
# def search_drug_image(image_path):
#     """이미지에서 약물을 검출하고 분류하여 CSV 형태로 반환"""
#     res = []
#     result = model_1way(image_path, verbose=False)

#     # 이미지 ID (파일명에서 추출)
#     image_id = os.path.splitext(os.path.basename(image_path))[0]

#     # 해당 이미지의 실제 numpy 배열 가져오기
#     current_img = result[0].orig_img.copy()

#     annotation_id = 1  # 각 이미지마다 1부터 시작

#     for box in result[0].boxes:
#         cls_idx, predicted_class, xyxy = search_class_idx(current_img, box)  # img → current_img
#         # print(f"Predicted class : [{cls_idx}]  {predicted_class} {xyxy}")

#         # YOLO bbox 좌표를 COCO 형식으로 변환
#         x1, y1, x2, y2 = xyxy
#         bbox_x = float(x1)
#         bbox_y = float(y1)
#         bbox_w = float(x2 - x1)
#         bbox_h = float(y2 - y1)

#         # confidence score
#         score = float(box.conf.item())

#         # category_id는 예측된 약물 코드
#         category_id = predicted_class

#         # CSV row 데이터
#         csv_row = [
#             annotation_id, image_id, category_id, bbox_x, bbox_y, bbox_w, bbox_h, score
#         ]

#         res.append(csv_row)
#         annotation_id += 1

#     return res

# def test_all_images_to_csv():
#     """모든 테스트 이미지에 대해 예측하고 CSV 형태로 저장"""
#     import pandas as pd

#     csv_rows = []
#     csv_headers = ['annotation_id', 'image_id', 'category_id', 'bbox_x', 'bbox_y', 'bbox_w', 'bbox_h', 'score']

#     print(f"총 {len(test_images)}개 이미지 처리 중...")

#     for i, image_path in enumerate(tqdm(test_images, desc="이미지 처리")):
#         try:
#             # 각 이미지의 예측 결과
#             image_results = search_drug_image(image_path)
#             csv_rows.extend(image_results)

#         except Exception as e:
#             print(f"이미지 처리 실패 {image_path}: {e}")
#             continue

#     # DataFrame 생성
#     df = pd.DataFrame(csv_rows, columns=csv_headers)

#     # CSV 파일로 저장
#     timestamp = datetime.now(__kst).strftime("%Y%m%d_%H%M%S")
#     filename = f'predictions_2way_model_{timestamp}.csv'
#     output_path = os.path.join(drive_root(),
#                                'codeit_ai_health_eat',
#                                'scripts',
#                                '김명환',
#                                filename)
#     df.to_csv(output_path, index=False)

#     print(f"\n예측 결과 저장 완료:")
#     print(f"- 총 이미지: {len(test_images)}개")
#     print(f"- 총 검출: {len(csv_rows)}개")
#     print(f"- 저장 경로: {output_path}")

#     # 결과 미리보기
#     print(f"\n결과 미리보기:")
#     print(df.head(10))

#     # 클래스별 통계
#     print(f"\n클래스별 검출 통계:")
#     class_counts = df['category_id'].value_counts().head(10)
#     print(class_counts)

#     return df, output_path

# # 실행
# results_df, csv_path = test_all_images_to_csv()

In [ ]:
# res = search_drug_image(test_images[0])

In [ ]:
# def test_all_images_to_csv():
#     """모든 테스트 이미지에 대해 예측하고 CSV 형태로 저장"""
#     for i, image_path in enumerate(tqdm(test_images, desc="이미지 처리")):
#         try:
#             # 각 이미지의 예측 결과
#             image_results = search_drug_image(image_path)
#         except Exception as e:
#             print(f"이미지 처리 실패 {image_path}: {e}")
#             continue

# test_all_images_to_csv()
